In [1]:
import cv2
import numpy as np

# Constants for disturbance tolerances
DISTURBANCE_TOLERANCE_UP = 10
DISTURBANCE_TOLERANCE_DOWN = -10
DISTURBANCE_TOLERANCE_HORI = 10

# Path to the Haar cascade file for face detection
HAAR_CASCADE_PATH = "haarcascade_frontalface_alt.xml"

# Load the cascade for face detection
cascade = cv2.CascadeClassifier(HAAR_CASCADE_PATH)

def detect_faces(image):
    # Detect faces in the image
    detected = cascade.detectMultiScale(image, scaleFactor=1.3, minNeighbors=4, flags=cv2.CASCADE_SCALE_IMAGE, minSize=(20, 20))
    print(detected)

    faces = []
    if len(detected) > 0:
        for (x, y, w, h) in detected:
            faces.append((x, y, w, h))
    return faces

def get_motion(faceCenter, originCenter):
    # Calculate horizontal and vertical changes
    horizontal_change = faceCenter[0] - originCenter[0]
    vertical_change = faceCenter[1] - originCenter[1]

    # Determine the direction of movement
    if abs(horizontal_change) < DISTURBANCE_TOLERANCE_HORI and abs(vertical_change) < DISTURBANCE_TOLERANCE_UP:
        print('ORIGIN', horizontal_change, vertical_change)
        return 25, 0
    elif horizontal_change > 0:
        print('LEFT', horizontal_change, vertical_change)
        return 1, horizontal_change
    elif horizontal_change < 0:
        print('RIGHT', horizontal_change, vertical_change)
        return 0, -horizontal_change
    elif vertical_change > 0:
        print('DOWN', horizontal_change, vertical_change)
        return 2, vertical_change
    elif vertical_change < 0:
        print('UP', horizontal_change, vertical_change)
        return 3, vertical_change

    print(horizontal_change, vertical_change)
    return None

def main(video_path):
    # Initialize video capture from the video file
    cap = cv2.VideoCapture(video_path)

    originCenter = None  # This will store the initial face position

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detect_faces(gray)

        if faces:
            x, y, w, h = faces[0]  # Consider the first detected face
            faceCenter = (x + w//2, y + h//2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            if originCenter is None:
                originCenter = faceCenter

            get_motion(faceCenter, originCenter)

        cv2.imshow('Face Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Specify the path to your video file here
    video_file_path = 'path_to_your_video.mp4'
    main(video_file_path)


OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!
